### Load Packages

In [95]:
import os
os.makedirs("icartt", exist_ok=True)
from __future__ import annotations
from dataclasses import dataclass
from typing import Dict, List, Optional, Tuple, Union
from pathlib import Path
import getpass
import requests
import pandas as pd
from zipfile import ZipFile
from datetime import datetime, timedelta

### Load ICT Converter

In [54]:
PathLike = Union[str, Path]


@dataclass(frozen=True)
class ICARTTInfo:
    path: Path
    header_length: int
    ffi: str


@dataclass(frozen=True)
class VariableDef:
    name: str
    unit: Optional[str] = None
    description: Optional[str] = None
    missing: Optional[float] = None  # per-variable missing, if known


class ICARTTReader:
    """
    General ICARTT/ICT reader.

    Goals:
      - Robustly read the data table (CSV-like) for typical ICT files.
      - Avoid file-specific assumptions (campaign/platform/column names).
      - Provide best-effort metadata parsing (especially for FFI=1001) but
        never let metadata parsing prevent data extraction.

    Notes:
      - Many airborne ICT files are FFI=1001 (1D time series), but other FFIs exist.
      - Header length is always the first token on the first line in the files you've shown.
    """

    def __init__(self, path: PathLike):
        self.path = Path(path)
        self.info = self._read_info()

    # ----------------------------
    # Core: file format info
    # ----------------------------
    def _read_info(self) -> ICARTTInfo:
        with open(self.path, "r", encoding="utf-8", errors="ignore") as f:
            line1 = f.readline().strip()

        parts = [p.strip() for p in line1.split(",")]
        if len(parts) < 2:
            raise ValueError(f"Unexpected ICARTT first line format: {line1!r}")

        header_length = int(parts[0])
        ffi = parts[1]
        return ICARTTInfo(path=self.path, header_length=header_length, ffi=ffi)

    def read_header_lines(self) -> List[str]:
        """Return the raw header lines (including line 1)."""
        n = self.info.header_length
        lines: List[str] = []
        with open(self.path, "r", encoding="utf-8", errors="ignore") as f:
            for _ in range(n):
                line = f.readline()
                if not line:
                    break
                lines.append(line.rstrip("\n"))
        return lines

    # ----------------------------
    # Minimal assumptions: table extraction
    # ----------------------------
    def read_table(
        self,
        *,
        na_values: Optional[List[Union[str, float, int]]] = None,
        strip_colnames: bool = True,
    ) -> pd.DataFrame:
        """
        Extract the data table.

        Strategy:
          - Most ICARTT files place the column header row at line `header_length`.
          - So we skip `header_length - 1` lines and let pandas treat the next line as header.

        This is general and does not depend on campaign/platform.
        """
        skiprows = max(self.info.header_length - 1, 0)

        # Many ICT files use -9999, -99999, etc., but we won't assume; allow caller to pass.
        # We'll also attempt to auto-detect common missing indicators from header if possible.
        if na_values is None:
            na_values = self._guess_missing_values()

        df = pd.read_csv(
            self.path,
            skiprows=skiprows,
            sep=",",
            encoding = "latin-1",
            encoding_errors = "ignore",
            engine="python",
            na_values=na_values,
        )

        if strip_colnames:
            df.columns = [str(c).strip() for c in df.columns]

        return df

    # ----------------------------
    # Best-effort metadata parsing
    # ----------------------------
    def read_metadata(self) -> Dict[str, str]:
        """
        Best-effort metadata extraction.

        Returns a dict of key metadata fields when the header matches common ICARTT layouts.
        If parsing fails, returns what it can without throwing.
        """
        lines = self.read_header_lines()
        meta: Dict[str, str] = {}

        # Common ICARTT: line indices below assume a conventional layout often used with FFI=1001.
        # We'll guard everything with length checks.
        def safe(i: int) -> str:
            return lines[i].strip() if 0 <= i < len(lines) else ""

        meta["path"] = str(self.path)
        meta["header_length"] = str(self.info.header_length)
        meta["ffi"] = self.info.ffi

        # These are common but not guaranteed. Keep them best-effort.
        meta["pi"] = safe(1)
        meta["organization"] = safe(2)
        meta["data_description"] = safe(3)
        meta["mission"] = safe(4)
        meta["volume_info"] = safe(5)
        meta["date_info"] = safe(6)
        meta["data_interval"] = safe(7)
        meta["independent_variable"] = safe(8)
        meta["seconds"] = safe(9)

        return {k: v for k, v in meta.items() if v}

    def read_variable_defs(self) -> List[VariableDef]:
        """
        Best-effort variable definitions, primarily for common FFI=1001 layout:
          - line 10: number of dependent variables
          - line 12+: variable definition lines (often "NAME, UNIT, DESCRIPTION...")

        If layout doesn't match, returns empty list.
        """
        lines = self.read_header_lines()

        # Attempt the common ICARTT/FFI=1001 positions
        # Line 10 (0-index 9) is often number of dependent variables.
        if len(lines) < 11:
            return []

        try:
            n_dep = int(lines[9].strip())
        except Exception:
            return []

        start = 12  # 0-index start of var definition block in common layout
        block = lines[start : start + n_dep]
        out: List[VariableDef] = []

        for ln in block:
            parts = [p.strip() for p in ln.split(",")]
            if not parts:
                continue
            name = parts[0]
            unit = parts[1] if len(parts) > 1 else None
            desc = ",".join(parts[2:]).strip() if len(parts) > 2 else None
            out.append(VariableDef(name=name, unit=unit or None, description=desc or None))

        # Attach per-variable missing if we can infer it (optional)
        miss_map = self._guess_per_variable_missing()
        if miss_map:
            out = [
                VariableDef(v.name, v.unit, v.description, miss_map.get(v.name))
                for v in out
            ]

        return out

    # ----------------------------
    # Missing-value handling
    # ----------------------------
    def _guess_missing_values(self) -> List[Union[str, float, int]]:
        """
        Heuristic: try to extract missing indicators from the header.
        Falls back to common sentinel values.

        Many ICT files have a line describing missing indicators (often around line 12),
        but formats vary. We keep this conservative.
        """
        lines = self.read_header_lines()
        candidates: List[Union[str, float, int]] = []

        # Scan header for something that looks like a missing indicator list: "-9999" etc.
        for ln in lines[: min(len(lines), 200)]:
            # find numeric sentinels that look like -9999, -99999, 9999 etc.
            for tok in ln.replace(",", " ").split():
                if tok.startswith(("-", "+")) and tok[1:].isdigit():
                    val = int(tok)
                    # common missing sentinels are large magnitude
                    if abs(val) >= 999:
                        candidates.append(val)

        # De-duplicate while preserving order
        seen = set()
        ordered = []
        for v in candidates:
            if v not in seen:
                seen.add(v)
                ordered.append(v)

        # Add very common defaults if we found nothing
        if not ordered:
            ordered = [-9999, -99999, -8888, 9999, 99999]

        return ordered

    def _guess_per_variable_missing(self) -> Dict[str, float]:
        """
        Some ICARTT headers specify per-variable missing indicators.
        This is not standardized across all producers; implement only as a best-effort hook.

        Returns {} if nothing reliable is found.
        """
        # For now, keep minimal: many files effectively use a single sentinel across columns.
        # You can extend this if you encounter a known pattern you want to support.
        return {}

    # ----------------------------
    # Exports
    # ----------------------------
    def to_csv(
        self,
        out: Optional[PathLike] = None,
        *,
        na_values: Optional[List[Union[str, float, int]]] = None,
        strip_colnames: bool = True,
    ) -> Path:
        df = self.read_table(na_values=na_values, strip_colnames=strip_colnames)
        out_path = Path(out) if out else self.path.with_suffix(".csv")
        df.to_csv(out_path, index=False)
        return out_path

    def to_parquet(
        self,
        out: Optional[PathLike] = None,
        *,
        na_values: Optional[List[Union[str, float, int]]] = None,
        strip_colnames: bool = True,
    ) -> Path:
        df = self.read_table(na_values=na_values, strip_colnames=strip_colnames)
        out_path = Path(out) if out else self.path.with_suffix(".parquet")
        df.to_parquet(out_path, index=False)
        return out_path

open("icartt/_init_.py", "w").close()

### Log Into NASA Session

In [15]:
home = Path.home()
netrc_path = home / "_netrc"

username = input("Enter username: ")
password = getpass.getpass("Enter password: ")

content = f"machine urs.earthdata.nasa.gov login {username} password {password}"

with open(netrc_path, "w") as f:
    f.write(content)

os.chmod(netrc_path, 0o600)

print(f".netrc file created at {netrc_path}")

file_path = home / ".urs_cookies"
file_path.touch(exist_ok=True)
print(f"Created: {file_path}")

session = requests.Session()
session.auth = None  # requests will use your .netrc automatically

auth_url = "http://asdc.larc.nasa.gov/soot-api/Authenticate/user"

Enter username: me.gonzalez674
Enter password: ········
.netrc file created at C:\Users\megon\_netrc
Created: C:\Users\megon\.urs_cookies


### Get List of Campaigns/File Names

In [90]:
#campaigns
base_url = "https://asdc.larc.nasa.gov/soot-api/campaigns"
response = requests.get(base_url)
response.raise_for_status()
campaign_json = response.json()
campaign_table = pd.DataFrame(campaign_json)

#list of years for given campaign
with pd.option_context('display.max_rows', None, 'display.max_columns', None,'display.max_colwidth', -1):
    display(campaign_table[["projectacronym", "description", "projecttitles"]])
campaign = input("Choose Campaign: ")
index = int(campaign_table[campaign_table["projectacronym"] == campaign].index.values)

campaign_specification = campaign_table.iloc[index]["projectacronym"] #choose campaign name
url = f'{base_url}/years/{campaign_specification}'
response = requests.get(url)
response.raise_for_status()
years_for_campaign_json = response.json()
years_for_campaign_table = pd.DataFrame(years_for_campaign_json)

#platforms for given year for given campaign
with pd.option_context('display.max_rows', None, 'display.max_columns', None,'display.max_colwidth', -1):
    display(years_for_campaign_table)
year = str(input("Choose Year: "))
index = int(years_for_campaign_table[years_for_campaign_table["year"] == year].index.values)

year_specification = years_for_campaign_table.iloc[index]["year"] #choose year
url = f'{base_url}/years/{campaign_specification}/{year_specification}'
response = requests.get(url)
response.raise_for_status()
platforms_for_year_json = response.json()
platforms_for_year_table = pd.DataFrame(platforms_for_year_json)

#PI for given platform for given year for given campaign
with pd.option_context('display.max_rows', None, 'display.max_columns', None,'display.max_colwidth', -1):
    display(platforms_for_year_table)
platform = str(input("Choose Platform (name NOT platformtype - DO NOT CHOOSE SATELLITE): "))
index = int(platforms_for_year_table[platforms_for_year_table["name"] == platform].index.values)

platform_specification = platforms_for_year_table.iloc[index]["name"] #choose platform name
url = f'{base_url}/years/{campaign_specification}/{year_specification}/{platform_specification}'
response = requests.get(url)
response.raise_for_status()
pi_for_platform_json = response.json()
pi_for_platform_table = pd.DataFrame(pi_for_platform_json)

#file names for given PI for given platform for given year for given campaign
with pd.option_context('display.max_rows', None, 'display.max_columns', None,'display.max_colwidth', -1):
    display(pi_for_platform_table[["investigatorid", "firstname", "lastname"]])
pi = str(input("Choose PI Last Name (Copy Exactly): "))
index = int(pi_for_platform_table[pi_for_platform_table["lastname"] == pi].index.values)

pi_specification = pi_for_platform_table.iloc[index]["lastname"]
url = f'{base_url}/years/{campaign_specification}/{year_specification}/{platform_specification}/{pi_specification}'
response = requests.get(url)
response.raise_for_status()
file_names_for_pi_json = response.json()
file_names_for_pi_table = pd.DataFrame(file_names_for_pi_json)

,projectacronym,description,projecttitles
0,ACCLIP,"The Asian Summer Monsoon Chemical & Climate Impact Project (ACCLIP) was an international, multi-organizational suborbital campaign that aimed to study aerosols and chemical transport that is associated with the Asian Summer Monsoon (ASM) in the Western Pacific region from 15 July 2022 to 31 August 2022. The ASM is the largest meteorological pattern in the Northern Hemisphere (NH) during the summer and is associated with persistent convection and large anticyclonic flow patterns in the upper troposphere and lower stratosphere (UTLS). This leads to significant enhancements in the UTLS of trace species that originate from pollution or biomass burning. Convection connected to the ASM occurs over South, Southeast, and East Asia, a region with complex and rapidly changing emissions due to its high population density and economic growth. Pollution that reaches the UTLS from this region can have significant effects on the climate and chemistry of the atmosphere, making it important to have an accurate representation and understanding of ASM transport, chemical, and microphysical processes for chemistry-climate models to characterize these interactions and for predicting future impacts on climate.\r\r The ACCLIP campaign was conducted by the National Aeronautics and Space Administration (NASA) and the National Center for Atmospheric Research (NCAR) with the primary goal of investigating the impacts of Asian gas and aerosol emissions on global chemistry and climate. The NASA WB-57 and NCAR G-V aircraft were outfitted with state-of-the-art sensors to accomplish this. ACCLIP addressed four scientific objectives related to its main goal. The first was to investigate the transport pathways of ASM uplifted air from inside of the anticyclone to the global UTLS. Another objective was sampling the chemical content of air processed in the ASM in order to quantify the role of the ASM in transporting chemically active species and short-lived climate forcing agents to the UTLS to determine their impact on stratospheric ozone chemistry and global climate. Third, information was obtained on aerosol size, mass, and chemical composition that is necessary for determining the radiative effects of the ASM to constrain models of aerosol formation and for contrasting the organic-rich ASM UTLS aerosol population with that of the background aerosols. Last, ACCLIP measured the water vapor distribution associated with the monsoon dynamical structure to evaluate transport across the tropopause and determine the role of the ASM in water vapor transport in the stratosphere.",[2022]
1,ACEPOL,"In order to improve our understanding of the effect of aerosols on climate and air quality, measurements of aerosol chemical composition, size distribution, height profile, and optical properties are of crucial importance. In terms of remotely sensed instrumentation, the most extensive set of aerosol properties can be obtained by combining passive multi-angle, multi-spectral measurements of intensity and polarization with active measurements performed by a High Spectral Resolution Lidar. During Fall 2017, the Aerosol Characterization from Polarimeter and Lidar (ACEPOL) campaign, jointly sponsored by NASA and the Netherlands Institute for Space Research (SRON), performed aerosol and cloud measurements over the United States from the NASA high altitude ER-2 aircraft. Six instruments were deployed on the aircraft. Four of these instruments were multi-angle polarimeters: the Airborne Hyper Angular Rainbow Polarimeter (AirHARP), the Airborne Multiangle SpectroPolarimetric Imager (AirMSPI), the Airborne Spectrometer for Planetary Exploration (SPEX Airborne) and the Research Scanning Polarimeter (RSP). The other two instruments were lidars: the High Spectral Resolution Lidar 2 (HSRL-2) and the Cloud Physics Lidar (CPL). The ACEPOL operation was based at NASA’s Armstrong Flight Research Center in Palmdale California, which enabled ob

Choose Campaign: DISCOVERAQ


,year,projectacronym
0,2011-MD,DISCOVERAQ
1,2013-CA,DISCOVERAQ
2,2013-TX,DISCOVERAQ
3,2014-CO,DISCOVERAQ


Choose Year: 2014-CO


,name,platformtype
0,ANALYSIS,Other
1,B200,Aircraft
2,FALCON,Aircraft
3,GROUND-AURORA-EAST,Ground Station
4,GROUND-BAO-TOWER,Ground Station
5,GROUND-BOULDER,Ground Station
6,GROUND-CAMP,Ground Station
7,GROUND-CHATFIELD-PARK,Ground Station
8,GROUND-DENVER-LACASA,Ground Station
9,GROUND-FORT-COLLINS,Ground Station


Choose Platform (name NOT platformtype - DO NOT CHOOSE SATELLITE): SONDES


,investigatorid,firstname,lastname
0,41,ANNE,THOMPSON


Choose PI Last Name (Copy Exactly): THOMPSON


### Download Files for Selected PI/Platform/Year/Campaign

In [96]:
session = requests.Session()
session.auth = None  # requests will use your .netrc automatically

auth_url = "http://asdc.larc.nasa.gov/soot-api/Authenticate/user"

for file in file_names_for_pi_table['filename']:
    get_file_url = f"http://asdc.larc.nasa.gov/soot-api/data_files/downloadFiles?filenames={file}"
    zip_file_name = get_file_url.split('=')[-1].split('.ict')[0] #get last part of filename
    
    response = session.get(auth_url)
    if response.status_code == 200:
        response = session.get(get_file_url)
        if response.status_code == 200:
            with open(f'{zip_file_name}.zip', 'wb') as f:
                f.write(response.content)
        else:
            print('ERROR: Unable to download files. Response code {response.status_code}')
    else:
        print('ERROR: User not authorized')
    
    #unzip the file you downloaded
    with ZipFile(f"{os.getcwd()}" + "\\" + f"{zip_file_name}.zip", 'r') as zObject:
        zObject.extractall()
    
    #delete the zip file, only keep the unzipped files
    os.remove(f"{zip_file_name}.zip")

### Convert Downloaded Files to CSVs and Delete ICT Files

In [112]:
folder = os.getcwd()

file_paths = []
for entry in os.scandir(folder):
    if entry.is_file():
        if entry.path[-3:] == "ict":
            file_paths += [entry.path]

format = '%Y,%m,%d'
        
combined_df = pd.DataFrame()
for file in file_paths:
    r = ICARTTReader(file)
    df = r.read_table()
    meta = r.read_metadata()
    vars_ = r.read_variable_defs()
    
    #find start date/time (only first 3 values)
    s = meta.get("date_info").split(',')
    s = ','.join(s[0:3])
    s = s.replace(" ", "")
    
    start_date = datetime.strptime(s, format)
    start_time = timedelta(seconds = int(meta.get("seconds")))
    start_datetime = start_date + start_time
    
    #find columns that have UTC seconds (Start_UTC, Seconds_UTC, etc.)
    time_columns = [col for col in df.columns if "UTC" in col.upper()]
    
    #create new column with full date listed
    for col in time_columns:
        new_col_name = col.replace("UTC", "Datetime")
        df[new_col_name] = start_datetime + pd.to_timedelta(df[col], unit = "s")
    
    #some data types have Time instead of UTC
    new_time_columns = []
    if len(time_columns) == 0:
        new_time_columns = [col for col in df.columns if "TIME" in col.upper()]
    
    for col in new_time_columns:
        column = col.title()
        new_col_name = column.replace("Time", "Datetime")
        df[new_col_name] = start_datetime + pd.to_timedelta(df[col], unit = "s")
   
    #combine dataframes
    combined_df = pd.concat([combined_df, df], ignore_index = True)

out_path = Path(f"{folder}\\{campaign}_{year}_{platform}_{pi}.csv")
csv_path = combined_df.to_csv(out_path)

#remove ICT files, only csvs left
for file in file_paths:
    file_name = file.split("\\")[-1]
    os.remove(file_name)